# Benchmarking with Our Test Split

This notebook details how you can benchmark with our test split using any function.

## Installs and Imports

In [14]:
# !curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.rpm.sh | sudo bash
# !sudo yum install git-lfs -y
# !git lfs install

In [2]:
%%capture
%pip install torch==1.9.0+cpu torchvision==0.10.0+cpu torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html
# %pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html
%pip install datasets==1.11.0
%pip install transformers==4.9.1
%pip install jiwer
%pip install ipywidgets
%pip install pythainlp==2.3.1
%pip install pydub
%pip install SpeechRecognition
%pip install azure-cognitiveservices-speech

In [110]:
from functools import partial
import pandas as pd
import numpy as np
from datasets import (
    load_dataset, 
    load_from_disk,
    load_metric,)
from datasets.filesystems import S3FileSystem
from transformers import (
    Wav2Vec2CTCTokenizer, 
    Wav2Vec2FeatureExtractor,
    Wav2Vec2Processor,
    Wav2Vec2ForCTC,
    TrainingArguments,
    Trainer,
)
import torch
import torchaudio
import re
import json
from pythainlp.tokenize import word_tokenize, syllable_tokenize
from tqdm.auto import tqdm

## Download Dataset

We download the whole Common Voice 7.0 even though we will use only our test split for benchmarking. We also copy `train_cleaned.tsv`, `dev_cleaned.tsv`, and `test_cleaned.tsv` for our cleaned and resampled splits.

In [111]:
# !wget https://voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com/cv-corpus-7.0-2021-07-21/cv-corpus-7.0-2021-07-21-th.tar.gz
# !tar -xvf cv-corpus-7.0-2021-07-21-th.tar.gz --no-same-owner
# !mv cv-corpus-7.0-2021-07-21-th ../data
# !cp train_cleaned.tsv dev_cleaned.tsv test_cleaned.tsv ../data/cv-corpus-7.0-2021-07-21-th/th

## Load Test Set

In [112]:
test_dataset = load_dataset("../scripts/th_common_voice_70.py", "th", split="test")
test_dataset

Reusing dataset common_voice (/home/ec2-user/.cache/huggingface/datasets/common_voice/th/7.0.0/14bf435a174687b310ed94f56abf0198f6cc7efb5a5d945c22c83113eab67701)


Dataset({
    features: ['path', 'sentence'],
    num_rows: 2502
})

## Load Metrics

We use WER with words tokenized by PyThaiNLP 2.3.1 and CER.

In [113]:
from pythainlp.tokenize import word_tokenize

wer_metric = load_metric("wer")
cer_metric = load_metric("cer")

## Preprocess

In [114]:
def speech_file_to_array_fn(batch, 
                            text_col="sentence", 
                            fname_col="path",
                            resampling_to=16000):
    speech_array, sampling_rate = torchaudio.load(batch[fname_col])
    resampler=torchaudio.transforms.Resample(sampling_rate, resampling_to)
    batch["speech"] = resampler(speech_array)[0].numpy()
    batch["sampling_rate"] = resampling_to
    batch["target_text"] = batch[text_col]
    return batch

test_dataset = test_dataset.map(speech_file_to_array_fn)

Loading cached processed dataset at /home/ec2-user/.cache/huggingface/datasets/common_voice/th/7.0.0/14bf435a174687b310ed94f56abf0198f6cc7efb5a5d945c22c83113eab67701/cache-b12df0a78d616663.arrow


## Evaluate

### Convert to `.wav` for Some APIs

We need to convert the `mp3` files to `wav`. Install `ffmpeg and ffprobe`:

In [115]:
# sudo su -
# mkdir -v -p /usr/local/bin/ffmpeg
# cd /usr/local/bin/ffmpeg
# wget https://johnvansickle.com/ffmpeg/releases/ffmpeg-release-i686-static.tar.xz
# tar -v -xf ffmpeg-release-i686-static.tar.xz --strip-components=1
# rm -v -f ffmpeg-release-i686-static.tar.xz
# ln -snf /usr/local/bin/ffmpeg/ffmpeg /usr/bin/ffmpeg
# ln -snf /usr/local/bin/ffmpeg/ffprobe /usr/bin/ffpropbe
# exit

In [116]:
from pydub import AudioSegment

def mp3_to_wav(example):
    example['path_wav'] = (example['path'][:-3] + 'wav').replace('clips','clips_wav')
    sound = AudioSegment.from_mp3(example['path'])
    sound.export(example['path_wav'], format="wav")
    return example

In [117]:
test_dataset = test_dataset.map(mp3_to_wav)
test_dataset

Loading cached processed dataset at /home/ec2-user/.cache/huggingface/datasets/common_voice/th/7.0.0/14bf435a174687b310ed94f56abf0198f6cc7efb5a5d945c22c83113eab67701/cache-1b78b0456d5d1b4f.arrow


Dataset({
    features: ['path', 'sentence', 'speech', 'sampling_rate', 'target_text', 'path_wav'],
    num_rows: 2502
})

### [Google Web Speech API](https://w3c.github.io/speech-api/speechapi.html)

In [118]:
import speech_recognition as sr

r = sr.Recognizer()

def evalute_example(fname):
    with sr.AudioFile(fname) as source:
        audio_text = r.listen(source)
    text = r.recognize_google(audio_text, language = "th-TH")
    return text
        
evalute_example(test_dataset[0]['path_wav'])

'และเขาก็สัมผัสดีบุก'

In [ ]:
ds = []
for i in tqdm(test_dataset):
    try:
        pred = evalute_example(i['path_wav']).replace(' ','')
    except:
        pred = ''
    d = {'sentence': i['sentence'].replace(' ',''), 
         'pred_sentence': pred}
    d['sentence_tok'] = ' '.join(word_tokenize(d['sentence']))
    d['pred_sentence_tok'] = ' '.join(word_tokenize(d['pred_sentence']))
    ds.append(d)

  0%|          | 0/2502 [00:00<?, ?it/s]

In [64]:
# result_df = pd.DataFrame(ds)
# result_df.to_csv('artifacts/result_google.csv',index=False)

result_df = pd.read_csv('artifacts/result_google.csv').fillna('')
result_df

,sentence,pred_sentence,sentence_tok,pred_sentence_tok
0,และเขาก็สัมผัสดีบุก,และเขาก็สัมผัสดีบุก,และ เขา ก็ สัมผัส ดีบุก,และ เขา ก็ สัมผัส ดีบุก
1,คุณสามารถรับทราบเมื่อข้อความนี้ถูกอ่านแล้ว,คุณสามารถรับทราบข้อความนี้ถูกอ่านแล้ว,คุณ สามารถ รับทราบ เมื่อ ข้อความ นี้ ถูก อ่าน ...,คุณ สามารถ รับทราบ ข้อความ นี้ ถูก อ่าน แล้ว
2,คืนนี้ฉันต้องทำให้ได้เธอพูดกับตัวเองฉันต้องทำใ...,คืนนี้ฉันต้องทำให้ได้เธอพูดกับตัวเองฉันต้องทำใ...,คืนนี้ ฉัน ต้อง ทำ ให้ได้ เธอ พูด กับ ตัวเอง ฉ...,คืนนี้ ฉัน ต้อง ทำ ให้ได้ เธอ พูด กับ ตัวเอง ฉ...
3,การทำเช่นนี้จะทำให้แผ่นถ่ายภาพเกิดแสงฟลูออเรสเ...,การทำเช่นนี้จะทำให้แผ่นถ่ายภาพเกิดแสงตัวartists,การ ทำ เช่นนี้ จะ ทำให้ แผ่น ถ่ายภาพ เกิด แสง ...,การ ทำ เช่นนี้ จะ ทำให้ แผ่น ถ่ายภาพ เกิด แสง ...
4,ผู้ปกครองของฉันไม่สนใจความเห็นนี้อย่างละเอียดอ...,ผู้ปกครองของฉันไม่สนใจความเห็นนี้อย่างละเอียดอ...,ผู้ปกครอง ของ ฉัน ไม่ สนใจ ความเห็น นี้ อย่าง ...,ผู้ปกครอง ของ ฉัน ไม่ สนใจ ความเห็น นี้ อย่าง ...
...,...,...,...,...
2497,ใครสนใจไปเรียนได้ฟรีวันละชั่วโมงสัปดาห์เดียวก็จบ,ใครสนใจจะเรียนได้ฟรีวันละชั่วโมงสัปดาห์เดียวก็จบ,ใคร สนใจ ไป เรียน ได้ ฟรี วัน ละ ชั่วโมง สัปดา...,ใคร สนใจ จะ เรียน ได้ ฟรี วัน ละ ชั่วโมง สัปดา...
2498,หรือในกรณีตั้งครรภ์เมื่ออายุเกิน,หรือในกรณีตั้งครรภ์เมื่ออายุเกิน,หรือ ใน กรณี ตั้งครรภ์ เมื่อ อายุ เกิน,หรือ ใน กรณี ตั้งครรภ์ เมื่อ อายุ เกิน
2499,ก่อนสเต็ปนั้น,ก่อนสเต็ปนั้น,ก่อน สเต็ป นั้น,ก่อน สเต็ป นั้น
2500,ถ้าจะแบนมันต้องชัดว่าคำพูดจะนำไปสู่เหตุการณ์พว...,ถ้าจะแบนมันต้องชัดว่าคำพูดจะนำไปสู่เหตุการณ์พว...,ถ้า จะ แบน มัน ต้อง ชัด ว่า คำพูด จะ นำไปสู่ เ...,ถ้า จะ แบน มัน ต้อง ชัด ว่า คำพูด จะ นำไปสู่ เ...


In [65]:
#wer
wer_metric.compute(predictions=result_df.pred_sentence_tok,references=result_df.sentence_tok)

0.1371123407540857

In [66]:
#cer
cer_metric.compute(predictions=result_df.pred_sentence,references=result_df.sentence)

0.07357340720221607

### [Microsoft Bing Speech API](https://azure.microsoft.com/en-us/services/cognitive-services/speech/)

In [108]:
import azure.cognitiveservices.speech as speechsdk

speech_config = speechsdk.SpeechConfig(subscription="", 
                                       region="")
# speech_config.speech_recognition_language = "th-TH"

audio_input = speechsdk.AudioConfig(filename=test_dataset[100]['path_wav'])
speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_input)
result = speech_recognizer.recognize_once_async().get()
result.text, result.error_json

RuntimeError: Runtime exception

In [ ]:
ds = []
for i in tqdm(test_dataset):
    try:
        pred = evalute_example(i['path_wav']).replace(' ','')
    except:
        pred = ''
    d = {'sentence': i['sentence'].replace(' ',''), 
         'pred_sentence': pred}
    d['sentence_tok'] = ' '.join(word_tokenize(d['sentence']))
    d['pred_sentence_tok'] = ' '.join(word_tokenize(d['pred_sentence']))
    ds.append(d)

In [ ]:
result_df = pd.DataFrame(ds)
result_df

In [ ]:
#wer
wer_metric.compute(predictions=result_df.pred_sentence_tok,references=result_df.sentence_tok)

In [ ]:
#cer
cer_metric.compute(predictions=result_df.pred_sentence,references=result_df.sentence)

### [NECTEC AI for Thai Partii API](https://aiforthai.in.th/aiplatform/#/speechtotext)

In [107]:
import requests
 
url = "https://api.aiforthai.in.th/partii-webapi"
 
fname = test_dataset[0]['path_wav']
files = {'wavfile': (fname, open(fname, 'rb'), 'audio/wav')}
 
headers = {
    'Apikey': "",
    'Cache-Control': "no-cache",
    'Connection': "keep-alive",
    }
 
param = {"outputlevel":"--uttlevel","outputformat":"--txt"}
 
response = requests.request("POST", url, headers=headers, files=files, data=param)
 
print("Result = " + response.text)

Result = {"status":"False","message":"No asr result","inputfilename":"","outputfilename":""}


In [70]:
fname

'../data/cv-corpus-7.0-2021-07-21/th/clips_wav/common_voice_th_25656242.wav'

### `wav2vec2` Models from HuggingFace

In [19]:
processor = Wav2Vec2Processor.from_pretrained("airesearch/wav2vec2-large-xlsr-53-th")
model = Wav2Vec2ForCTC.from_pretrained("airesearch/wav2vec2-large-xlsr-53-th")

def evaluate(batch):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    inputs = processor(batch["speech"], sampling_rate=16_000, return_tensors="pt", padding=True)

    with torch.no_grad():
        logits = model(inputs.input_values.to(device),).logits

    pred_ids = torch.argmax(logits, dim=-1)
    batch["pred_sentence"] = processor.batch_decode(pred_ids)
    return batch

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [25]:
#takes quite a long time on CPU
result = test_dataset.map(evaluate, batched=True, batch_size=8)
result_df = pd.DataFrame({'sentence':result['sentence'], 'pred_sentence_tok': result['pred_sentence']})
result_df['sentence_tok'] = result_df.sentence.map(lambda x: ' '.join(word_tokenize(x)))
result_df['pred_sentence'] = result_df.pred_sentence_tok.map(lambda x: ''.join(x.split()))
result_df

In [26]:
#wer
wer_metric.compute(predictions=result_df.pred_sentence_tok,references=result_df.sentence_tok)

In [27]:
#cer
cer_metric.compute(predictions=result_df.pred_sentence,references=result_df.sentence)